# 11-1 데이터 집계

In [5]:
#데이터 집계하기
import pandas as pd
df = pd.read_csv('C:/Users/admin/Desktop/데이터 분석을 위한 판다스 입문/doit_pandas-master/data/gapminder.tsv', sep = '\t')
avg_life_exp_by_year = df.groupby('year').lifeExp.mean()
print(avg_life_exp_by_year)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


In [7]:
#평균값을 구하는 사용자 함수와 groupby 메서드
def my_mean(values):
    n = len(values)
    sum = 0
    for value in values:
        sum = sum + value
    return sum / n

agg_my_mean = df.groupby('year').lifeExp.agg(my_mean)
print(agg_my_mean)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


In [13]:
#2개의 인잣값을 받아 처리하는 사용자 함수와 groupby 메서드
def my_mean_diff(values, diff_value):
    n = len(values)
    sum = 0
    for value in values:
        sum = sum + value
    mean = sum / n
    return mean - diff_value

global_mean = df.lifeExp.mean()
print(global_mean)
print()
agg_my_mean_diff = df.groupby('year').lifeExp.agg(my_mean_diff, diff_value = global_mean)    #두번째 요소인 diff_value는 따로 입력
print(agg_my_mean_diff)

59.47443936619713

year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64


In [16]:
#여러 개의 집계 메서드 한 번에 사용하기
import numpy as np
gdf = df.groupby('year').lifeExp.agg([np.count_nonzero, np.mean, np.std])
print(gdf)
print()
gdf_dict = df.groupby('year').agg({'lifeExp': 'mean', 'pop': 'median', 'gdpPercap': 'median'})
print(gdf_dict)

      count_nonzero       mean        std
year                                     
1952          142.0  49.057620  12.225956
1957          142.0  51.507401  12.231286
1962          142.0  53.609249  12.097245
1967          142.0  55.678290  11.718858
1972          142.0  57.647386  11.381953
1977          142.0  59.570157  11.227229
1982          142.0  61.533197  10.770618
1987          142.0  63.212613  10.556285
1992          142.0  64.160338  11.227380
1997          142.0  65.014676  11.559439
2002          142.0  65.694923  12.279823
2007          142.0  67.007423  12.073021

        lifeExp         pop    gdpPercap
year                                    
1952  49.057620   3943953.0  1968.528344
1957  51.507401   4282942.0  2173.220291
1962  53.609249   4686039.5  2335.439533
1967  55.678290   5170175.5  2678.334741
1972  57.647386   5877996.5  3339.129407
1977  59.570157   6404036.5  3798.609244
1982  61.533197   7007320.0  4216.228428
1987  63.212613   7774861.5  4280.300366
1

# 11-2 데이터 변환

In [18]:
#표준점수 계산하기
def my_zscore(x):
    return (x - x.mean()) / x.std()

transform_z = df.groupby('year').lifeExp.transform(my_zscore)
print(transform_z.head())

0   -1.656854
1   -1.731249
2   -1.786543
3   -1.848157
4   -1.894173
Name: lifeExp, dtype: float64


In [30]:
# 누락값을 평균값으로 처리하기
import seaborn as sns
import numpy as np

np.random.seed(42)
tips_10 = sns.load_dataset('tips').sample(10)
tips_10.loc[np.random.permutation(tips_10.index)[:4], 'total_bill'] = np.NaN     #total_bill 중 4개의 값을 랜덤하게 누락값으로 변경
print(tips_10)
print()

count_sex = tips_10.groupby('sex').count()     #남녀별 누락값 수 확인
print(count_sex)
print()

def fill_na_mean(x):
    avg = x.mean()
    return x.fillna(avg)     #평균으로 누락값 대체

total_bill_group_mean = tips_10.groupby('sex').total_bill.transform(fill_na_mean)
tips_10['fill_total_bill'] = total_bill_group_mean
print(tips_10)

     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153         NaN  2.00    Male     No   Sun  Dinner     4
211         NaN  5.16    Male    Yes   Sat  Dinner     4
198         NaN  2.00  Female    Yes  Thur   Lunch     2
176         NaN  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2

        total_bill  tip  smoker  day  time  size
sex                                             
Male             4    7       7    7     7     7
Female           2    3       3    3     3     3

     total_bill   tip     sex smoker   day    time  size  fill_total_bill
24        19.82  3.18    Male     No   Sat  Dinner     2          19.8200
6          8.77  2.00    Ma

# 11-3 데이터 필터링

In [36]:
#데이터 필터링
tips = sns.load_dataset("tips")
print(tips.shape)
print()

print(tips['size'].value_counts())
print()

tips_filtered = tips.groupby('size').filter(lambda x: x['size'].count() >= 30)
print(tips_filtered['size'].value_counts())

(244, 7)

2    156
3     38
4     37
5      5
6      4
1      4
Name: size, dtype: int64

2    156
3     38
4     37
Name: size, dtype: int64


# 11-4 그룹 오브젝트

In [41]:
#여러 열을 사용해 그룹 오브젝트 만들기
bill_sex_time = tips_10.groupby(['sex', 'time'])
group_avg = bill_sex_time.mean()

print(group_avg)

               total_bill       tip      size  fill_total_bill
sex    time                                                   
Male   Lunch    28.440000  2.560000  2.000000        28.440000
       Dinner   14.456667  2.928333  2.666667        16.204583
Female Lunch    12.480000  2.260000  2.000000        13.205000
       Dinner   15.380000  3.000000  2.000000        15.380000
